## Moduliai

In [1]:
%load_ext nb_black

import pandas as pd
import numpy as np
import glob
import os
import sys

from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpgrowth
from mlxtend.frequent_patterns import association_rules

from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import MinMaxScaler

from scipy.sparse import csr_matrix


from IPython.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

# pd.set_option("mode.chained_assignment", None)

<IPython.core.display.Javascript object>

## Duomenų skaitymas, tvarkymas

In [16]:
def read_txt(filename: str):
    df = pd.read_csv(
        filename,
        sep="\t",
        #     nrows=100000,
        #         thousands=".",
        #         decimal=",",
        #         usecols=range(1, 16),
        dtype={
            1: str,
            2: str,
            3: str,
            4: str,
            5: str,
            7: str,
            8: str,
            9: str,
            10: str,
            11: str,
        },
        encoding="utf-8",
    )
    return df

<IPython.core.display.Javascript object>

In [17]:
path = r".\data"  # use your path
all_files = glob.glob(path + "/*.txt")

# ls = [read_txt(file) for file in all_files]
# df0 = pd.concat(ls)
df0 = read_txt(all_files[0])

<IPython.core.display.Javascript object>

In [18]:
print(df0.shape)
df0.head()

(204124, 11)


,kvitas,nom_id,artikulas,preke,parduotuve,pirkejo_id,kategorija,kat_id,brendas,brendo_id,tipas
0,G12021971,0028025,_EKOK018100F,*Vienkartinės kepuraitės 1/100vnt.,"Kaunas, Molas, Cascada",A13319,Grozis - nebegaminami,0023542,Eko Higiena,00294,Pardavimas per EKA
1,G12021971,0027487,EKOK05380CM8,Paklodė rulone 80cm/80m,"Kaunas, Molas, Cascada",A13319,Vienkartinės paklodės,0052186,Eko Higiena,00294,Pardavimas per EKA
2,G14003366,0023747,SIMR18,Natūralus žalias vaškas kasetėje 100ml.,"Kaunas, Mega, Cascada",A69846,Vaškas kasetėse,0052002,Simple Use Beauty,00174,Pardavimas per EKA
3,G14003366,0023746,SIMR20,Natūralus geltonas vaškas kasetėje 100ml.,"Kaunas, Mega, Cascada",A69846,Vaškas kasetėse,0052002,Simple Use Beauty,00174,Pardavimas per EKA
4,G03003491,0023178,NKG,Aktyvacijos mokestis NK grožio prekėms,"Kaunas, Akropolis, Cascada",C096837,Nepaskirstytos,0052243,Nuolaidų Kortelės,04881,Pardavimas per EKA


<IPython.core.display.Javascript object>

In [19]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 204124 entries, 0 to 204123
Data columns (total 11 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   kvitas      204124 non-null  object
 1   nom_id      204124 non-null  object
 2   artikulas   204124 non-null  object
 3   preke       204124 non-null  object
 4   parduotuve  204124 non-null  object
 5   pirkejo_id  110373 non-null  object
 6   kategorija  204124 non-null  object
 7   kat_id      204124 non-null  object
 8   brendas     204120 non-null  object
 9   brendo_id   204120 non-null  object
 10  tipas       204124 non-null  object
dtypes: object(11)
memory usage: 17.1+ MB


<IPython.core.display.Javascript object>

Unikalių prekių

In [20]:
df0["nom_id"].value_counts().count()

10035

<IPython.core.display.Javascript object>

Dokumentų tipai

In [21]:
df0["tipas"].value_counts()

Pardavimas per EKA       203763
Grąžinimas iš pirkėjo       361
Name: tipas, dtype: int64

<IPython.core.display.Javascript object>

Triname grąžinimus, bei eilutes be kvito numerio, prekes, kurios yra "Maišeliai ir dėžutės" bei "Nuolaidų Kortelės" kategorijoje. 

In [22]:
df1 = df0[df0["tipas"] != "Grąžinimas iš pirkėjo"]
df1 = df1[~df1["kvitas"].isna()]
df1 = df1[~(df1["kat_id"] == "0042490")]  # "Maišeliai ir dėžutės"
df1 = df1[~(df1["kat_id"] == "0052243")]  # "Nuolaidų kortelės"

<IPython.core.display.Javascript object>

Ištriname nereikalingus stulpelius

In [23]:
df2 = df1.copy()

df2 = df2.drop(["artikulas", "tipas", "parduotuve"], axis=1)
df = df2[~df2.duplicated()]

<IPython.core.display.Javascript object>

In [24]:
pardavimai = df[["kvitas", "nom_id"]]

pardavimai = pardavimai.groupby("kvitas")["nom_id"].apply(list).reset_index()

prekes = df[["nom_id", "preke"]].copy()
prekes = prekes[~prekes.duplicated()]
prekes = prekes.set_index("nom_id")
prekes = prekes["preke"]

<IPython.core.display.Javascript object>

In [28]:
pardavimai

,kvitas,nom_id
0,G02283546,"[0013947, 0041230, 0041236]"
1,G02283547,"[0008612, 0011242, 0019252, 0029366]"
2,G02283549,"[0029364, 0019324, 0018087, 0041236]"
3,G02283551,"[0029367, 0041135, 0027486, 0027487]"
4,G02283553,"[0013147, 0033868, 0048810, 0041673, 0022577]"
...,...,...
89522,G41061099,"[0010851, 0023349]"
89523,G41061100,[0025027]
89524,G41061101,"[0025234, 0027497]"
89525,G41061102,[0051556]


<IPython.core.display.Javascript object>

## Duomenų enkodingas

In [29]:
te = TransactionEncoder()

te.fit(pardavimai["nom_id"])

pardavimai_1hot = te.transform(pardavimai["nom_id"])
pardavimai_1hot = pd.DataFrame(pardavimai_1hot, columns=te.columns_)
# pardavimai_1hot.head()

<IPython.core.display.Javascript object>

## Apriori

In [57]:
%%timeit -n1 -r1
apriori(pardavimai_1hot, min_support=0.005, max_len=2, use_colnames=True)

2.31 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


<IPython.core.display.Javascript object>

In [58]:
is_ap = apriori(pardavimai_1hot, min_support=0.005, max_len=2, use_colnames=True)

<IPython.core.display.Javascript object>

In [59]:
print(len(is_ap))
is_ap.head()

29


,support,itemsets
0,0.011985,(0008608)
1,0.022016,(0008612)
2,0.011739,(0010851)
3,0.005105,(0011242)
4,0.005317,(0012219)


<IPython.core.display.Javascript object>

In [60]:
is_ap[is_ap["itemsets"].apply(lambda x: len(x)) == 2]

,support,itemsets
28,0.009327,"(0048513, 0048514)"


<IPython.core.display.Javascript object>

## FP Growth

In [67]:
%%timeit -n1 -r4
fpgrowth(pardavimai_1hot, min_support=0.001, max_len=2, use_colnames=True)

1.84 s ± 27.3 ms per loop (mean ± std. dev. of 4 runs, 1 loop each)


<IPython.core.display.Javascript object>

In [68]:
is_fp = fpgrowth(pardavimai_1hot, min_support=0.001, max_len=4, use_colnames=True)

<IPython.core.display.Javascript object>

In [69]:
print(is_fp.shape)
is_fp[is_fp["itemsets"].apply(lambda x: len(x)) == 2]

(408, 2)


,support,itemsets
384,0.001854,"(0041230, 0041236)"
385,0.001653,"(0041241, 0041236)"
386,0.003016,"(0008612, 0019252)"
387,0.001441,"(0019252, 0008608)"
388,0.001452,"(0011242, 0008612)"
389,0.001028,"(0011242, 0019252)"
390,0.001028,"(0025027, 0010851)"
391,0.001262,"(0025234, 0027491)"
392,0.001407,"(0019247, 0019251)"
393,0.001530,"(0019252, 0019251)"


<IPython.core.display.Javascript object>

## Parodysime kad `is_ap` ir `is_fp` turi tas pačias reikšmes

In [63]:
def itemset_to_ordered_string(itemset):
    return ",".join(sorted(list(itemset)))

<IPython.core.display.Javascript object>

In [64]:
ap_itemset_strings = is_ap["itemsets"].apply(itemset_to_ordered_string)
ap_itemset_strings = ap_itemset_strings.sort_values().reset_index(drop=True)
ap_itemset_strings.head()

0    0008608
1    0008612
2    0010851
3    0011242
4    0012219
Name: itemsets, dtype: object

<IPython.core.display.Javascript object>

In [65]:
fp_itemset_strings = is_fp["itemsets"].apply(itemset_to_ordered_string)
fp_itemset_strings = fp_itemset_strings.sort_values().reset_index(drop=True)
fp_itemset_strings.head()

0    0008608
1    0008612
2    0010851
3    0011242
4    0012219
Name: itemsets, dtype: object

<IPython.core.display.Javascript object>

In [66]:
fp_itemset_strings.equals(ap_itemset_strings)

True

<IPython.core.display.Javascript object>

## Asociacijų taisyklės

In [72]:
rules = association_rules(is_fp, metric="lift", min_threshold=0.8)
print(len(rules))
rules.sort_values(by="support", ascending=False)

48


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43,(0048514),(0048513),0.012923,0.014968,0.009327,0.721694,48.217240,0.009133,3.539387
42,(0048513),(0048514),0.014968,0.012923,0.009327,0.623134,48.217240,0.009133,2.619173
27,(0038471),(0008612),0.006881,0.022016,0.004211,0.612013,27.798928,0.004060,2.520662
26,(0008612),(0038471),0.022016,0.006881,0.004211,0.191273,27.798928,0.004060,1.228004
22,(0008612),(0008608),0.022016,0.011985,0.004032,0.183156,15.281813,0.003768,1.209551
23,(0008608),(0008612),0.011985,0.022016,0.004032,0.336440,15.281813,0.003768,1.473844
38,(0041230),(0041241),0.009941,0.005708,0.003250,0.326966,57.284366,0.003194,1.477329
39,(0041241),(0041230),0.005708,0.009941,0.003250,0.569472,57.284366,0.003194,2.299637
5,(0019252),(0008612),0.011293,0.022016,0.003016,0.267062,12.130537,0.002767,1.334335
4,(0008612),(0019252),0.022016,0.011293,0.003016,0.136986,12.130537,0.002767,1.145645


<IPython.core.display.Javascript object>

## Prognozės (predictions)

In [73]:
def predict(antecedent, rules, max_results=5):
    preds = rules[rules["antecedents"] == antecedent]
    preds = preds["consequents"].apply(iter).apply(next)
    return preds.iloc[:max_results].reset_index(drop=True)

<IPython.core.display.Javascript object>

In [74]:
prekes_kodas = "0008612"
preds = predict({prekes_kodas}, rules)

print(f"{prekes_kodas:8} - {prekes[prekes_kodas]}")
print("-" * 40)
for stockid in preds:
    print(f"{stockid:8} - {prekes[stockid]}")

0008612  - Blakstienų ir antakių dažai, Nr.3, rudi 
----------------------------------------
0019252  - Oksidantas, 3%, 50ml 
0011242  - Kosmetininiai šepetėliai, 1/5vnt 
0008608  - Blakstienų ir antakių dažai, Nr.1, juodi 
0038471  - Oksidacinė emulsija, 3%, 10 vol.,100 ml 
0038468  - Oksidacinis skystis, 3%, 10 vol., 100 ml 


<IPython.core.display.Javascript object>

In [75]:
prekes_kodas = "0019252"
preds = predict({prekes_kodas}, rules)

print(f"{prekes_kodas:8} - {prekes[prekes_kodas]}")
print("-" * 40)
for stockid in preds:
    print(f"{stockid:8} - {prekes[stockid]}")

0019252  - Oksidantas, 3%, 50ml 
----------------------------------------
0008612  - Blakstienų ir antakių dažai, Nr.3, rudi 
0008608  - Blakstienų ir antakių dažai, Nr.1, juodi 
0011242  - Kosmetininiai šepetėliai, 1/5vnt 
0019251  - Antakių/blakstienų dažai, juodi, 15ml 
0019247  - Antakių/blakstienų dažai, rudi, 15ml 


<IPython.core.display.Javascript object>

## Vartotojas-vartotojui (user-to-user collaborative filtering)

Vartotojai skaitomi panašiausi, jei jie perka panašias prekes. Tam panašumui įvertinti baudojama metrika _cosine_, t.y. mažiausias 'kampas' tarp dviejų vartotojų prekių reitingų
Sudarome reitingų lentelę naudojantis pirkimų skaičiumi

In [77]:
df_pvt = pd.pivot_table(
    df,
    values="kvitas",
    index=["pirkejo_id"],
    columns=["nom_id"],
    aggfunc=len,
    fill_value=0,
)
df_pvt1 = df_pvt.transpose()

<IPython.core.display.Javascript object>

Sunormuojame reitingus

In [78]:
# https://stackoverflow.com/questions/63463531/min-max-scaling-for-whole-dataframe-python
min_value, max_value = df_pvt1.min().min(), df_pvt1.max().max()
df_pvt_scaler = (df_pvt1 - min_value) * 10 / (max_value - min_value)


<IPython.core.display.Javascript object>

In [79]:
klientai = df_pvt1.columns
pr = df_pvt1.index
df_pvt2 = pd.DataFrame(df_pvt_scaler, columns=klientai, index=pr).T
df_pvt2

nom_id,0001010,0001345,0001346,0001351,0001442,0001477,0001478,0001654,0001718,0001721,...,0054795,0054803,0054804,0054805,0054872,0054873,0054878,0054887,0054905,0054938
pirkejo_id,,,,,,,,,,,,,,,,,,,,,
00135,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00168,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00178,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00195,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00202,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C155075,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C159295,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
C163766,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


<IPython.core.display.Javascript object>

In [80]:
ratings = df_pvt2.unstack().reset_index(name="rating")
ratings = ratings[ratings["rating"] > 0].reset_index(drop=True)
ratings.sort_values(by="rating")

MemoryError: Unable to allocate 2.74 GiB for an array with shape (2, 183558217) and data type object

<IPython.core.display.Javascript object>

In [81]:
ratings_cross = pd.crosstab(
    ratings.pirkejo_id, ratings.nom_id, ratings.rating, aggfunc=sum
).fillna(0)

R = csr_matrix(ratings_cross.values)
pirkejai = ratings_cross.reset_index(0)[["pirkejo_id"]]

MemoryError: Unable to allocate 2.74 GiB for an array with shape (2, 183558217) and data type object

<IPython.core.display.Javascript object>

In [82]:
pirkejai[pirkejai["pirkejo_id"] == "A41515"].index[0]

NameError: name 'pirkejai' is not defined

<IPython.core.display.Javascript object>

In [39]:
ratings.pirkejo_id.index[ratings.pirkejo_id == "A41515"]

Int64Index([ 4215, 26103, 26323, 26544, 26816, 27165, 27187, 30383, 43913,
            63566, 63962, 64472, 69436],
           dtype='int64')

<IPython.core.display.Javascript object>

Funkicja _nearest_neighbors()_ grąžina kiekvieno vartotojui artimiausių vartotojų sąrąša su atstumais

In [40]:
def create_model(rating_matrix, metric):
    """
    - create the nearest neighbors model with the corresponding similarity metric
    - fit the model
    """
    model = NearestNeighbors(metric=metric, n_neighbors=21, algorithm="brute")
    model.fit(rating_matrix)
    return model


def nearest_neighbors(rating_matrix, model):
    """
    :param rating_matrix : rating matrix of shape (nb_users, nb_items)
    :param model : nearest neighbors model
    :return
        - similarities : distances of the neighbors from the referenced user
        - neighbors : neighbors of the referenced user in decreasing order of similarities
    """
    similarities, neighbors = model.kneighbors(rating_matrix)
    return similarities[:, 1:], neighbors[:, 1:]

<IPython.core.display.Javascript object>

In [41]:
model = create_model(
    rating_matrix=R, metric="cosine"
)  # we can also use the 'euclidian' distance
similarities, neighbors = nearest_neighbors(R, model)

<IPython.core.display.Javascript object>

In [42]:
ratings.pirkejo_id

0          11078
1         A13709
2         A18899
3         A92689
4         A23895
          ...   
95185     A05224
95186    C042351
95187     A79835
95188     B28618
95189      06731
Name: pirkejo_id, Length: 95190, dtype: object

<IPython.core.display.Javascript object>

Randame prekes-kandidatus

In [43]:
def find_candidate_items(pirkejo_id):
    """
    Find candidate items for an active user

    :param userid : active user
    :param neighbors : users similar to the active user
    :return candidates : top 30 of candidate items
    """
    userid = pirkejai[pirkejai["pirkejo_id"] == pirkejo_id].index[0]
    user_neighbors = neighbors[userid]
    activities = ratings.loc[ratings.index.isin(user_neighbors)]
    # sort items in decreasing order of frequency

    frequency = (
        activities.groupby("nom_id")["rating"]
        .count()
        .reset_index(name="count")
        .sort_values(["count"], ascending=False)
    )
    Gu_items = frequency.nom_id
    active_items = ratings.loc[ratings.pirkejo_id == pirkejo_id].nom_id.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]

    return candidates

<IPython.core.display.Javascript object>

In [44]:
pirkejas = "A41515"
find_candidate_items(pirkejas)

array(['0012219', '0010921', '0014239', '0019444', '0019329', '0019239',
       '0018762', '0017377', '0016402', '0005814', '0006421', '0011763',
       '0011731', '0009735', '0008922', '0008612', '0008610', '0022448'],
      dtype=object)

<IPython.core.display.Javascript object>

Reitingo prognozavimas įvertinant vidutinį kiekvieno pirkėjo reitingą.

In [45]:
# mean ratings for each user
mean = ratings.groupby(by="pirkejo_id", as_index=False)["rating"].mean()
mean_ratings = pd.merge(ratings, mean, suffixes=("", "_mean"), on="pirkejo_id")

# normalized ratings for each items
mean_ratings["norm_rating"] = mean_ratings["rating"] - mean_ratings["rating_mean"]

mean = mean.to_numpy()[:, 1]
np_ratings = mean_ratings.to_numpy()

<IPython.core.display.Javascript object>

In [46]:
mean_ratings[
    (mean_ratings["pirkejo_id"] == pirkejas) & (mean_ratings["nom_id"] == "0025234")
]

,nom_id,pirkejo_id,rating,rating_mean,norm_rating
26536,0025234,A41515,0.588235,1.719457,-1.131222


<IPython.core.display.Javascript object>

In [47]:
np_ratings[:, 1]


array(['11078', '11078', '11078', ..., 'A67196', 'B51475', 'A59900'],
      dtype=object)

<IPython.core.display.Javascript object>

In [48]:
def predict(pirkejo_id, itemid):
    """
    predict what score userid would have given to itemid.

    :param
        - userid : user id for which we want to make prediction
        - itemid : item id on which we want to make prediction

    :return
        - r_hat : predicted rating of user userid on item itemid
    """
    userid = pirkejai[pirkejai["pirkejo_id"] == pirkejo_id].index[0]
    user_similarities = similarities[userid]
    user_neighbors = neighbors[userid]
    # get mean rating of user userid
    user_mean = mean[userid]
    #     print(f"{user_mean=}")

    # find users who rated item 'itemid'
    iratings = np_ratings[np_ratings[:, 1] == itemid]
    #     print(f"{iratings=}")

    # find similar users to 'userid' who rated item 'itemid'
    suri = iratings[np.isin(iratings[:, 0], user_neighbors)]
    #     print(f"{suri=}")
    # similar users who rated current item (surci)
    normalized_ratings = suri[:, 4]
    indexes = [
        np.where(user_neighbors == uid)[0][0] for uid in suri[:, 0].astype("int")
    ]
    sims = user_similarities[indexes]

    num = np.dot(normalized_ratings, sims)
    den = np.sum(np.abs(sims))

    if num == 0 or den == 0:
        return user_mean

    r_hat = user_mean + np.dot(normalized_ratings, sims) / np.sum(np.abs(sims))

    return r_hat

<IPython.core.display.Javascript object>

In [49]:
predict(pirkejas, "0025234")

1.7194570135746605

<IPython.core.display.Javascript object>

In [54]:
def user2userPredictions(pirkejo_id, pred_path):
    """
    Make rating prediction for the active user on each candidate item and save in file prediction.csv

    :param
        - userid : id of the active user
        - pred_path : where to save predictions
    """
    # find candidate items for the active user
    candidates = find_candidate_items(pirkejo_id)
    print(f"{candidates[0]=}")
    # loop over candidates items to make predictions
    for itemid in candidates:
        # prediction for userid on itemid
        r_hat = predict(pirkejo_id, itemid)
        print(f"{r_hat=}")
        # save predictions
        with open(pred_path, "a+") as file:
            line = "{},{},{}\n".format(pirkejo_id, itemid, r_hat)
            file.write(line)

<IPython.core.display.Javascript object>

In [55]:
def user2userCF():
    """
    Make predictions for each user in the database.
    """
    # get list of users in the database
    users = ratings.pirkejo_id.unique()

    def _progress(count):
        sys.stdout.write(
            "\rRating predictions. Progress status : %.1f%%\n"
            % (float(count / len(users)) * 100.0)
        )
        sys.stdout.flush()

    saved_predictions = "predictions.csv"
        if os.path.exists(saved_predictions):
            os.remove(saved_predictions)

    for count, pirkejo_id in enumerate(users[:5]):
        # make rating predictions for the current user
        user2userPredictions(pirkejo_id, saved_predictions)
        _progress(count)

IndentationError: unexpected indent (2947329643.py, line 16)

ERROR:root:Cannot parse: 16:0:         if os.path.exists(saved_predictions):
Traceback (most recent call last):
  File "C:\miniconda3\envs\code_academy\lib\site-packages\lab_black.py", line 218, in format_cell
    formatted_code = _format_code(cell)
  File "C:\miniconda3\envs\code_academy\lib\site-packages\lab_black.py", line 29, in _format_code
    return format_str(src_contents=code, mode=FileMode())
  File "C:\miniconda3\envs\code_academy\lib\site-packages\black\__init__.py", line 1079, in format_str
    dst_contents = _format_str_once(src_contents, mode=mode)
  File "C:\miniconda3\envs\code_academy\lib\site-packages\black\__init__.py", line 1089, in _format_str_once
    src_node = lib2to3_parse(src_contents.lstrip(), mode.target_versions)
  File "C:\miniconda3\envs\code_academy\lib\site-packages\black\parsing.py", line 127, in lib2to3_parse
    raise exc from None
black.parsing.InvalidInput: Cannot parse: 16:0:         if os.path.exists(saved_predictions):


In [ ]:
user2userCF()

In [ ]:
def user2userRecommendation(userid):
    """ """
    # encode the userid
    #     uid = uencoder.transform([userid])[0]
    uid = userid
    saved_predictions = "predictions.csv"

    predictions = pd.read_csv(
        saved_predictions, sep=",", names=["userid", "itemid", "predicted_rating"]
    )
    predictions = predictions[predictions.userid == uid]
    List = predictions.sort_values(by=["predicted_rating"], ascending=False)
    print(uid)
    print(List)
    print(prekes)
    #     List.userid = uencoder.inverse_transform(List.userid.tolist())
    #     List.itemid = iencoder.inverse_transform(List.itemid.tolist())

    #     List = pd.merge(List, prekes, left_on="itemid", right_on="nom_id" how="inner")

    return List

In [ ]:
user2userRecommendation("00135")

In [ ]:
def find_candidate_items(userid):
    """
    Find candidate items for an active user
    
    :param userid : active user
    :param neighbors : users similar to the active user        
    :return candidates : top 30 of candidate items
    """
    user_neighbors = neighbors[userid]
    activities = ratings.loc[ratings.userid.isin(user_neighbors)]
    
    # sort items in decreasing order of frequency
    frequency = activities.groupby('itemid')['rating'].count().reset_index(name='count').sort_values(['count'],ascending=False)
    Gu_items = frequency.itemid
    active_items = ratings.loc[ratings.userid == userid].itemid.to_list()
    candidates = np.setdiff1d(Gu_items, active_items, assume_unique=True)[:30]
        
    return candidates